In [150]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever

In [3]:
# ## BGE Embeddings
# from langchain.embeddings import HuggingFaceBgeEmbeddings

# model_name = "BAAI/bge-large-en-v1.5"
# encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
# device = "cpu"
# bge_embeddings = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': device},
#     encode_kwargs=encode_kwargs
# )

.gitattributes: 100%|██████████| 1.52k/1.52k [00:00<00:00, 89.2kB/s]
1_Pooling/config.json: 100%|██████████| 191/191 [00:00<?, ?B/s] 
README.md: 100%|██████████| 90.3k/90.3k [00:00<00:00, 343kB/s]
config_sentence_transformers.json: 100%|██████████| 124/124 [00:00<?, ?B/s] 
model.safetensors: 100%|██████████| 1.34G/1.34G [01:42<00:00, 13.0MB/s]
pytorch_model.bin: 100%|██████████| 1.34G/1.34G [02:14<00:00, 9.98MB/s]
sentence_bert_config.json: 100%|██████████| 52.0/52.0 [00:00<00:00, 8.22kB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 8.50kB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 9.50MB/s]
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 36.1kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.26MB/s]
modules.json: 100%|██████████| 349/349 [00:00<00:00, 50.8kB/s]


In [151]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
device = "cpu"
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': device},
    encode_kwargs=encode_kwargs
)

In [152]:
print(len(bge_embeddings.embed_query("hello world, iron man")))

384


### Data prep

In [37]:

# from langchain.document_loaders import DirectoryLoader
# from langchain.document_loaders import TextLoader
# loader = DirectoryLoader('doc_txt/', glob="**/*.text", 
#                          loader_cls=TextLoader, 
#                          use_multithreading=True,
#                          show_progress=True)
# docs = loader.load()


In [153]:
from bs4 import BeautifulSoup
import json
import os

vectors = []

folder = "patent_jsons"
files = [os.path.join(folder, f) for f in os.listdir(folder) ]

print(files[1])
print(len(files))

patent_jsons\JP-H10177520-A.json
500


In [154]:
print(files[1][13:-5])

JP-H10177520-A


In [155]:
for i,path in enumerate(files):
    if i>=0:
        data = {"id":f"{i}",
                "values":None, 
                "metadata":{"file_name": path[13:-5],
                            "title":None, 
                            "lang":None}}

        with open(path, "r") as read_file:
            doc = json.load(read_file)
            # print(len(doc["descriptions"]))
            # if not os.path.exists("doc_txt"):
            #     os.mkdir("doc_txt")
            for file in doc["descriptions"]:
                if file["lang"]=="EN":
                    data["metadata"]["lang"]="EN"
                    html_data = file["paragraph_markup"]
                    # Extract the paragraph_markup from the data

                    soup = BeautifulSoup(html_data, 'html.parser')

                    # Extract all text from the HTML structure
                    all_text = soup.get_text(separator='\n', strip=True)
                    # data["metadata"]["content"]=all_text
                    
                    # with open(f"doc_txt/doc_{i}.text", "w") as write_file:
                    #     write_file.write(all_text)
                    
            # if not os.path.exists("title_txt"):
            #     os.mkdir("title_txt")
            for title in doc["titles"]:
                if title["lang"]=="EN":
                    title_data = title["text"]
                    
                    # with open(f"title_txt/title_{i}.text", "w") as write_file:
                    #     write_file.write(title_data)
        data["metadata"]["title"]=title_data
        data["values"] = bge_embeddings.embed_query(title_data+all_text)
        vectors.append(data)          
    else:
        break

Upserted vectors:   0%|          | 0/500 [2:29:41<?, ?it/s]


In [156]:
print(len(vectors))

500


### creating pinecone db

In [157]:
import pinecone
from langchain.vectorstores import Pinecone

In [158]:
# import dotenv
# dotenv.load_dotenv()
import os 
# api_key = os.getenv("PINECONE_API_KEY")

pinecone.init(api_key="c9fa65e7-443c-4c12-bea1-0b835c561d9d", environment="gcp-starter")

In [159]:
index = pinecone.Index('first-index')

In [160]:
index.upsert(vectors=vectors, batch_size=50, show_progress=True)

Upserted vectors: 100%|██████████| 500/500 [00:07<00:00, 63.92it/s]


{'upserted_count': 500}

# 1. Retrieving full documents rather than chunks

In [165]:
query1 = "Data processing apparatus having high speed slave store and multi-word instruction buffer"
query_emb = bge_embeddings.embed_query(query1)

In [166]:
# import pinecone 

# pinecone.init(api_key='YOUR_API_KEY', environment='us-east1-gcp') 
# index = pinecone.Index('example-index') 

query_response = index.query(
    # namespace='example-namespace',
    top_k=5,
    include_values=False,
    include_metadata=True,
    vector=query_emb,
    # sparseVector={
    #     'indices': [10, 45, 16],
    #     'values':  [0.5, 0.5, 0.2]
    # },
    # filter={
    #     'genre': {'$in': ['comedy', 'documentary', 'drama']}
    # }
)

In [167]:
print(len(query_response["matches"]))

5


In [168]:
print(query_response["matches"])

[{'id': '8',
 'metadata': {'file_name': 'US-3949376-A',
              'lang': 'EN',
              'title': 'Data processing apparatus having high speed slave '
                       'store and multi-word instruction buffer'},
 'score': 0.903763354,
 'values': []}, {'id': '53',
 'metadata': {'file_name': 'US-5179680-A',
              'lang': 'EN',
              'title': 'Instruction storage and cache miss recovery in a high '
                       'speed multiprocessing parallel processing apparatus'},
 'score': 0.801680863,
 'values': []}, {'id': '49',
 'metadata': {'file_name': 'US-5123097-A',
              'lang': 'EN',
              'title': 'APPARATUS AND METHOD FOR SIMULTANEOUS EXECUTION OF A '
                       'WRITE INSTRUCTION AND A SUCCEEDING READ INSTRUCTION IN '
                       'A DATA PROCESSING SYSTEM WITH A STORE THROUGH CACHE '
                       'STRATEGY'},
 'score': 0.793522835,
 'values': []}, {'id': '212',
 'metadata': {'file_name': 'US-5860107-A'

---

In [171]:
folder = "patent_jsons"
files = [os.path.join(folder, f) for f in os.listdir(folder) ]

db_patent = "patent_files.json"
if not os.path.exists(db_patent):
    db = {"data":{}}
    with open(db_patent, "w") as write_file:
        json.dump(db, write_file, indent=4)

    for i,path in enumerate(files):
        if i>=0:
            data = {"html_data": None, "title":None}
            with open(path, "r") as read_file:
                doc = json.load(read_file)
                for file in doc["descriptions"]:
                    if file["lang"]=="EN":
                        html_data = file["paragraph_markup"]


                for title in doc["titles"]:
                    if title["lang"]=="EN":
                        title_data = title["text"]

            data["title"]=title_data
            data["html_data"] = html_data

            with open(db_patent, "r") as read_file:
                db_loaded = json.load(read_file)
                db_loaded["data"][path[13:-5]]=data

            with open(db_patent, "w") as write_file:
                json.dump(db_loaded, write_file, indent=4)
        else:
            break
